# 전처리

### 파이썬 결측치 대체 평균

In [1]:
import pandas as pd

train = pd.read_csv(".data/train.csv")
test = pd.read_csv(".data/test.csv")

In [43]:
train_null = list(zip(train.columns.to_list(), train.isnull().sum()))
for col, n in train_null:
	if n:
		train.fillna({col:int(train[col].mean())}, inplace=True)

### interpolate() method

따릉이 데이터의 경우, feature들은 기상정보들이며 데이터의 순서는 시간 순서이다. 그렇기에 결측치들을 이전 행(직전 시간)과 이후 행(직후 시간)의 평균으로 보간하는 것이 합리적이다. 그렇기에 interpolate()로 결측치를 대체해야 한다.

In [46]:
train = pd.read_csv(".data/train.csv")
test = pd.read_csv(".data/test.csv")

In [68]:
train_null = list(zip(train.columns.to_list(), train.isnull().sum()))
for i, n in train_null:
	if n: train.interpolate(inplace=True)
test.fillna(0, inplace=True)

# 모델링

### 랜덤포레스트 개념, 선언

``` 랜덤포레스트 ``` 는 여러 개의 의사결정나무를 만들어서 이들의 평균으로 예측의 성능을 높이는 방법이다. 이렇게 여러 개의 모델들을 이용해 성능을 높이는 기법을 ``` 앙상블(Ensemble) ``` 기법이라고 한다.

In [49]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

### 랜덤포레스트를 평가척도에 맞게 학습

랜덤포레스트의 여러 옵션 중 criterion 옵션은 어떠한 평가척도를 기준으로 훈련할 것인지 정할 수 있다.

따릉이 대회의 평가지표는 RMSE이다. RMSE는 MSE 평가지표에 루트를 씌운 것으로 모델을 선언할 때, ``` criterion='mse' ``` 옵션으로 구현할 수 있다.

$$ MSE = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y})^2 $$

In [50]:
model = RandomForestRegressor(criterion='mse')

In [51]:
X_train = train.drop(['count'], axis=1)
Y_train = train['count']

In [52]:
model.fit(X_train, Y_train)

C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse')

# 튜닝

### 랜덤포레스트 변수중요도 확인

fit()으로 학습된 모델은 feature_importances_ 속성(attribute)으로 변수의 중요도를 파악할 수 있다. 변수의 중요도란 예측 변수를 결정할 때, 각 피쳐가 얼마나 중요한 역할을 하는지에 대한 척도이다.

- 변수의 중요도: 예측변수를 결정할 때, 각 피쳐가 얼마나 중요한 역할을 하는지에 대한 척도

정의에 따라 변수의 중요도가 낮다면 해당 피쳐를 제거하는 것이 모델의 성능을 높일 수도 있다는 의미이다.

In [60]:
for c, i in zip(X_train.columns, model.feature_importances_):
	print(f"{c}: {i}")

id: 0.02519611982335789
hour: 0.5916669341062244
hour_bef_temperature: 0.17955279066338872
hour_bef_precipitation: 0.0189562829536924
hour_bef_windspeed: 0.025554485971880012
hour_bef_humidity: 0.03701949400517842
hour_bef_visibility: 0.03322718697564536
hour_bef_ozone: 0.036345174143937085
hour_bef_pm10: 0.032032037330242145
hour_bef_pm2.5: 0.02044949402645348


### 변수 제거

우선 id는 예측에 의미가 없다. 따라서 id를 제외한 데이터를 생성해야 한다. 또한 test 역시 train과 동일한 feature를 가져야 하기에 id를 제외한다. 그 외에도 위 결과를 보며 필요 없는 것들을 제거해본다.

In [70]:
X_train_1 = train.drop(['id', 'hour_bef_precipitation', 'count'], axis=1)
test_1 = test.drop(['id', 'hour_bef_precipitation'], axis=1)

model_1 = RandomForestRegressor(criterion='mse')
model_1.fit(X_train_1, Y_train)

C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mse')

In [71]:
pred_1 = model_1.predict(test_1)
submission_1 = pd.read_csv('.data/submission.csv')
submission_1['count'] = pred_1
submission_1.to_csv('sub_1.csv', index=False)

### 하이퍼파라미터 / GridSearch

하이퍼파라미터 튜닝은 정지규칙 값들을 설정하는 것을 의미한다. 의사결정나무에는 정지규칙(stopping criteria)라는 개념이 있다.

- **최대 깊이(max_depth)**
	- 최대로 내려갈 수 있는 depth
    - 작을수록 트리는 작아짐
- **최소 노드크기(min_samples_split)**
	- 노드를 분할하기 위한 데이터 수
	- 해당 노드에 이 값보다 작은 확률변수 수가 있다면 멈춤
    - 작을수록 트리는 커짐
-  **최소 향상도(min_impurity_split)**
	- 노드를 분할하기 위한 최소 향상도
    - 향상도가 설정값 이하라면 더 이상 분할하지 않음
    - 작을수록 트리는 커짐
- **비용 복잡도(cost-complexity)**
	- 트리가 커지는 것에 대해 패널티 계수를 설정해서 불순도와 트리가 커지는 것에 대해 복잡도를 계산하는 것

이와 같이 정지규칙들을 종합적으로 고려해 최적의 조건값을 설정할 수 있다. 이를 하이퍼파라미터 튜닝이라고 한다.

이러한 하이퍼파라미터 튜닝의 방법 중 하나가 **GridSearch**인데 완전 탐색으로 최적의 파라미터를 찾는 방법이다.

In [4]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv(".data/train.csv")
test = pd.read_csv(".data/test.csv")

train.interpolate(inplace=True)
test.fillna(0, inplace=True)

X_train = train.drop(['count', 'id', 'hour_bef_pm2.5', 'hour_bef_precipitation'], axis=1)
Y_train = train['count']
test = test.drop(['id', 'hour_bef_pm2.5', 'hour_bef_precipitation'], axis=1)

model = RandomForestRegressor(criterion='mse', random_state=2020)

params = {
	'n_estimators':[200, 300, 400, 500],
    'max_features':[5, 6, 7, 8],
    'min_samples_leaf':[1, 3, 4, 5]
}

greedy_CV = GridSearchCV(model, param_grid=params, cv=3, n_jobs=-1)
greedy_CV.fit(X_train, Y_train)

pred = greedy_CV.predict(test)
submission = pd.read_csv('.data/submission.csv')

import numpy as np

submission['count'] = np.round(pred, 2)
print(submission.head(3))
submission.to_csv('sub.csv', index=False)

C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
48 fits failed out of a total of 192.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):


   id   count
0   0   96.07
1   1  216.50
2   2   90.96
